# Get Time Series for the Data Cube

In [2]:
import sys
sys.path.append("../../helpers")

from data import Data
from maptools import MapTools
from widgets import Widgets

import ipywidgets as widgets
#import matplotlib
#matplotlib.use('nbagg')
from IPython.core.display import HTML

display(HTML("""
<style>
.container { 
    width:60% !important; 
}
</style>
"""))

display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))

%matplotlib inline

def on_button_clicked(b): 
    button.disabled = True
    d = Data(out)
    
    if num_prods.value == 1:
        data = d.data_to_csv(product1.value,
                      subproduct1.value,
                      latitude.value,
                      longitude.value,
                      start.value,
                      end.value)
    else:
        data = d.data_to_csv(product1.value,
                      subproduct1.value,
                      latitude.value,
                      longitude.value,
                      start.value,
                      end.value,
                      product2.value,
                      subproduct2.value)     
        
    button.disabled = False
    
m = MapTools(center=(1, -5), zoom=3, width='380px', height='500px')
m.map.add_control(m.dc)
m.prepare_map(rect=False)

w = Widgets()
num_prods = w.number_of_products(options=[1, 2])
start, end = w.get_date_widgets()
latitude, longitude = w.get_lat_lon_widgets()
product1 = w.product("Product 1:", "product1")
subproduct1 = w.subproduct("Subproduct 1:", "subproduct1")
product2 = w.product("Product 2:", "product2")
subproduct2 = w.subproduct("Subproduct 2:", "subproduct2")
button = w.set_up_button(on_button_clicked, 'Get csv')
w.display_widgets_csv(num_prods, product1, subproduct1, product2, subproduct2, 
                      latitude, longitude, start, end, button, m.map)

label = widgets.Label()
m.mouse_interaction(label)

def update_lat(x):
    """"
    Updates the latitude coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (latitude, 'value'))

def update_lon(x):
    """"
    Updates the longitude coordinate display by linking widgets. 
    
    :param x: update value
    """
    w = MapTools.update_nesw(x)
    l = widgets.link((w.children[0], 'value'), (longitude, 'value'))

    
def handle_draw(self, action, geo_json):
    """
    Handles drawing of layers on the map by extracting coordinates
    and updating the relevant coordinate display box.
    
    :param action: 
    :param geo_json: 
    
    :return:
    """
    w = Widgets()
    d = Data(out)
    
    s = geo_json.get('geometry','type')
    t = s.get('type')
    
    if t == 'Point':
        north, east, south, west = MapTools.get_coords_point(geo_json)
        update_lat(north)
        update_lon(east)


    if t == 'Polygon':
        pass

# Define a function that updates the content of y based on what we select for x
def update(*args):
    subproduct1.options = w.get_subproduct_list(product1.value)
    subproduct2.options = w.get_subproduct_list(product2.value)
    
# Updates the selection of product/subproducts
def update_number(*args):
    if num_prods.value == 1:
        product2.layout.visibility = 'hidden'
        subproduct2.layout.visibility = 'hidden'
        
    elif num_prods.value == 2:
        product2.layout.visibility = 'visible'
        subproduct2.layout.visibility = 'visible'     

update()
update_number()
num_prods.observe(update_number)
product1.observe(update)
product2.observe(update)


out = w.display_output()

m.dc.on_draw(handle_draw)

Label(value='')

Output()